In [1]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from verstack import NaNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
# from imblearn.over_sampling import RandomOverSampler

In [8]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline

In [9]:
df = pd.read_csv('loan_level_500k.csv')
df.head()

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_STATISTICAL_AREA,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN_TO_VALUE,ORIGINAL_DEBT_TO_INCOME_RATIO,...,PROPERTY_TYPE,POSTAL_CODE,LOAN_SEQUENCE_NUMBER,LOAN_PURPOSE,ORIGINAL_LOAN_TERM,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,PREPAID,DELINQUENT
0,669.0,200206,N,202901,NaN,0.0,1.0,O,80.0,33.0,...,SF,26100.0,F199Q1000004,P,320,2.0,Other sellers,Other servicers,True,False
1,732.0,199904,N,202903,17140.0,0.0,1.0,O,25.0,10.0,...,SF,45200.0,F199Q1000005,N,360,1.0,Other sellers,Other servicers,True,False
2,679.0,200208,N,202902,15940.0,30.0,1.0,O,91.0,48.0,...,SF,44700.0,F199Q1000007,P,319,1.0,Other sellers,Other servicers,True,False
3,721.0,200209,N,202902,38060.0,0.0,1.0,O,39.0,13.0,...,SF,85200.0,F199Q1000013,N,318,2.0,Other sellers,Other servicers,True,False
4,618.0,200210,N,202902,10420.0,25.0,1.0,O,85.0,24.0,...,SF,44200.0,F199Q1000015,N,317,2.0,Other sellers,Other servicers,True,False


In [10]:
df.drop('LOAN_SEQUENCE_NUMBER', inplace=True, axis =1)

In [11]:
df.shape

(500137, 26)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500137 entries, 0 to 500136
Data columns (total 26 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   CREDIT_SCORE                      497426 non-null  float64
 1   FIRST_PAYMENT_DATE                500137 non-null  int64  
 2   FIRST_TIME_HOMEBUYER_FLAG         369578 non-null  object 
 3   MATURITY_DATE                     500137 non-null  int64  
 4   METROPOLITAN_STATISTICAL_AREA     429988 non-null  float64
 5   MORTGAGE_INSURANCE_PERCENTAGE     449089 non-null  float64
 6   NUMBER_OF_UNITS                   500134 non-null  float64
 7   OCCUPANCY_STATUS                  500137 non-null  object 
 8   ORIGINAL_COMBINED_LOAN_TO_VALUE   500124 non-null  float64
 9   ORIGINAL_DEBT_TO_INCOME_RATIO     485208 non-null  float64
 10  ORIGINAL_UPB                      500137 non-null  int64  
 11  ORIGINAL_LOAN_TO_VALUE            500128 non-null  f

In [13]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
CREDIT_SCORE,497426.0,712.536212,54.791262,300.000,676.000,719.0,756.000,839.0
FIRST_PAYMENT_DATE,500137.0,200025.430952,109.815541,199901.000,199904.000,200005.0,200105.000,201103.0
MATURITY_DATE,500137.0,203023.195872,110.384189,202402.000,202903.000,203004.0,203104.000,204101.0
METROPOLITAN_STATISTICAL_AREA,429988.0,30777.824739,11333.401144,10180.000,19740.000,33340.0,40420.000,49740.0
MORTGAGE_INSURANCE_PERCENTAGE,449089.0,7.744532,12.046546,0.000,0.000,0.0,18.000,55.0
NUMBER_OF_UNITS,500134.0,1.028890,0.218391,1.000,1.000,1.0,1.000,4.0
ORIGINAL_COMBINED_LOAN_TO_VALUE,500124.0,76.053571,15.139986,6.000,70.000,80.0,88.000,180.0
ORIGINAL_DEBT_TO_INCOME_RATIO,485208.0,32.917541,11.111800,1.000,25.000,33.0,41.000,65.0
ORIGINAL_UPB,500137.0,136493.484785,60968.743066,8000.000,89000.000,126000.0,176000.000,578000.0
ORIGINAL_LOAN_TO_VALUE,500128.0,75.710714,14.937717,6.000,70.000,80.0,85.000,100.0


Dropping irrelevant columns (Not Available during prediction)

In [14]:
df.drop(["FIRST_PAYMENT_DATE", "MATURITY_DATE", "MORTGAGE_INSURANCE_PERCENTAGE", "ORIGINAL_UPB", "ORIGINAL_INTEREST_RATE", "PREPAYMENT_PENALTY_MORTGAGE_FLAG","NUMBER_OF_BORROWERS"], inplace=True, axis=1)

In [15]:
print(df.isnull().sum().sort_values())

PRODUCT_TYPE                            0
SERVICER_NAME                           0
SELLER_NAME                             0
ORIGINAL_LOAN_TERM                      0
LOAN_PURPOSE                            0
PROPERTY_STATE                          0
PREPAID                                 0
CHANNEL                                 0
DELINQUENT                              0
OCCUPANCY_STATUS                        0
NUMBER_OF_UNITS                         3
ORIGINAL_LOAN_TO_VALUE                  9
ORIGINAL_COMBINED_LOAN_TO_VALUE        13
POSTAL_CODE                            31
PROPERTY_TYPE                          95
CREDIT_SCORE                         2711
ORIGINAL_DEBT_TO_INCOME_RATIO       14929
METROPOLITAN_STATISTICAL_AREA       70149
FIRST_TIME_HOMEBUYER_FLAG          130559
dtype: int64


In [16]:
def missing_percentage(df):
    missing = pd.DataFrame(columns=['Category', 'Percentage'])
    for col in df.columns:
        if df[col].isna().values.any():
            percentage = 100*df[col].isna().sum()/df.shape[0]
            missing = missing.append({'Category':col, 'Percentage':percentage}, ignore_index = True)
    return missing

In [17]:
missingdata = missing_percentage(df)
missingdata.sort_values('Percentage', ascending=False)

,Category,Percentage
1,FIRST_TIME_HOMEBUYER_FLAG,26.104647
2,METROPOLITAN_STATISTICAL_AREA,14.025957
5,ORIGINAL_DEBT_TO_INCOME_RATIO,2.984982
0,CREDIT_SCORE,0.542051
7,PROPERTY_TYPE,0.018995
8,POSTAL_CODE,0.006198
4,ORIGINAL_COMBINED_LOAN_TO_VALUE,0.002599
6,ORIGINAL_LOAN_TO_VALUE,0.0018
3,NUMBER_OF_UNITS,0.0006


In [18]:
df.FIRST_TIME_HOMEBUYER_FLAG.value_counts()

N    320418
Y     49160
Name: FIRST_TIME_HOMEBUYER_FLAG, dtype: int64

In [19]:
categorical = list(df.select_dtypes(include=['object','bool']).columns[:-1])
numerical = df.select_dtypes(include=['int64','float64']).columns[:-1]

In [20]:
df.drop('FIRST_TIME_HOMEBUYER_FLAG',axis = 1,  inplace=True)

Seperate Data

In [21]:
X_n = df.drop('DELINQUENT', axis =1)
y_n = df['DELINQUENT']

In [22]:
y_n = y_n*1

In [18]:
imputer = NaNImputer()
X_n = imputer.impute(X_n)

NaNImputer(conservative = False, n_feats = 10,            
           fix_string_nans = True, verbose = True,                
           multiprocessing_load = 3, fill_nans_in_pure_text = True,                    
           drop_empty_cols = True, drop_nan_cols_with_constant = True                        
           feature_selection = correlation)

Dataset dimensions:
 - rows:         500137
 - columns:      17
 - mb in memory: 61.53
 - NaN cols num: 8
--------------------------

Deploy multiprocessing with 12 parallel proceses


NaNs imputation time: 0.73 minutes
--------------------------------------------------


In [19]:
y_n.value_counts().to_frame().T

,0,1
DELINQUENT,482146,17991


Train test Split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_n, y_n, test_size=0.3, random_state=42)

Random Forest HyperParameters

In [24]:
# param_grid = {
#     'n_estimators':[200,500],
#     'max_features':['auto', 'sqrt', 'log2'],
#     'max_depth':[4,5,6,7,8],
#     'criterion':['gini', 'entropy']
# }

BaseEstimator and TransformerMixin Approach

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline as imbPipeline

In [27]:
class Impute(BaseEstimator, TransformerMixin):

    def fit(self,X, y=None):
        return self

    def transform(self, X):
        categorical = list(X.select_dtypes(include=['object','bool']).columns[:-1])
        numerical = X.select_dtypes(include=['int64','float64']).columns[:-1]
        for i in numerical:
            X[i] = X[i].fillna(X[i].median())
        for i in categorical:
            X[i] = X[i].fillna(X[i].mode()[0])
        return X

In [28]:
class FeatureEncoder(BaseEstimator, TransformerMixin):

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        label_encoder = LabelEncoder()
        X['PREPAID']= label_encoder.fit_transform(X['PREPAID'])
        X['POSTAL_CODE'] = label_encoder.fit_transform(X['POSTAL_CODE'])
        return X

In [29]:
class Ordinal_Encoder(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self,X):
        enc = OrdinalEncoder(handle_unknown='ignore')
        X = enc.fit_transform(X)
        return X

In [30]:
class Scaling(BaseEstimator, TransformerMixin):

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        scaler = StandardScaler(with_mean=False)
        X = scaler.fit_transform(X)
        return X

In [31]:
pipe = imbPipeline([
    ("Imputer", Impute()),
    ("Label Encoder", FeatureEncoder()),
    ("Encoder", OrdinalEncoder()),
    ("Scaling", Scaling()),
    ("Over sample", SMOTE()),
    ("Classifier", RandomForestClassifier())
])

In [ ]:
pipe.fit(X_train,y_train)

In [61]:
pipe.named_steps

{'Imputer': Impute(),
 'Label Encoder': FeatureEncoder(),
 'Encoder': OrdinalEncoder(),
 'Scaling': Scaling(),
 'Over sample': SMOTE(),
 'Classifier': RandomForestClassifier()}

In [62]:
from sklearn import set_config
set_config(display='diagram')

In [64]:
y_pred = pipe.predict(X_test)

ValueError: Found unknown categories [333.0, 391.0, 416.0, 432.0, 448.0, 453.0, 464.0, 466.0, 480.0, 832.0, 835.0] in column 0 during transform

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

Pipeline using Column TransFormer

In [ ]:
# from sklearn.base import TransformerMixin
#
#
# class MyStandardScaler(TransformerMixin):
#     def __init__(self, *args, **kwargs):
#         self.scaler = StandardScaler(*args,**kwargs)
#     def fit(self,x,y=0):
#         self.scaler.fit(x)
#         return self
#     def transform(self,x,y=0):
#         return self.scaler.transform(x)
#
#
#

In [ ]:
# Ordinal_features = ['OCCUPANCY_STATUS', 'CHANNEL', 'PRODUCT_TYPE', 'PROPERTY_STATE', 'PROPERTY_TYPE', 'LOAN_PURPOSE', 'SELLER_NAME', 'SERVICER_NAME']

In [ ]:
# trf1 = ColumnTransformer([
#     ('Ordinal Encoder', OrdinalEncoder(),Ordinal_features)
# ],remainder='passthrough')

In [ ]:
# trf2 = ColumnTransformer([
#     ('NaN Imputer',NaNImputer()),
#     ('Scalar', MyStandardScaler()),
#     ('Random Over Sample', RandomOverSampler())
# ])

In [ ]:
# trf3 = RandomForestClassifier()

In [ ]:
# pipe = make_pipeline(trf1, trf2, trf3)

In [ ]:
# pipe.fit(X_train, y_train)

Pipeline

In [ ]:
# Label_Enc = Pipeline(steps=[
#     ('Label Imputer', LabelEncoder())
# ])
#
# Ord_Enc = Pipeline(steps=[
#     ('Ordinal Encoder', OrdinalEncoder())
# ])
# All_transformer = Pipeline(steps=[
#     ('NaN Imputer',NaNImputer()),
#     ('Scalar', StandardScaler()),
#     ('Smote', RandomOverSampler())
# ])

In [ ]:
# Label_features = ['DELINQUENT','PREPAID','POSTAL_CODE']
# Ordinal_features = ['OCCUPANCY_STATUS', 'CHANNEL', 'PRODUCT_TYPE', 'PROPERTY_STATE', 'PROPERTY_TYPE', 'LOAN_PURPOSE', 'SELLER_NAME', 'SERVICER_NAME']
# All_features = X.select_dtypes(include=['int64', 'float64', 'object']).columns

In [ ]:
# preprocessor = ColumnTransformer([
#         ('Label', Label_Enc, Label_features),
#         ('Ordinal', Ord_Enc, Ordinal_features),
#         ('Data Processing', All_transformer, All_features)
# ])

In [ ]:
# trf1 = RandomForestClassifier()

In [ ]:
# pipe = make_pipeline(preprocessor,trf1)

In [ ]:
# pipe.fit(X_train, y_train)

In [ ]:
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('Classifier', RandomForestClassifier())
# ])

In [ ]:
# rf_model = pipeline.fit(X_train, y_train)
# print(rf_model)